In [6]:
import sys, os



%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import os
os.environ['R_HOME'] = 'D:/Program Files/R-4.5.0' #@TODO include in some setup file

from simulation_engine.scenarios.iv.binary_iv import BinaryIV
from simulation_engine.scenarios.conf.binary_conf import BinaryConf
from simulation_engine.scenarios.iv.continuous_iv import ContinuousIV
from simulation_engine.scenarios.customDAG.customBinaryDAG import CustomBinaryDAG
from simulation_engine.util.plotting_util import PlottingUtil
from simulation_engine.algorithms.causaloptim import Causaloptim
from simulation_engine.algorithms.zhang_bareinboim import ZhangBareinboim
from simulation_engine.algorithms.apid import Apid
from simulation_engine.algorithms.autobound import AutoBound
from simulation_engine.algorithms.apid_src.src.models.apid import APID

import pandas as pd
pd.set_option('display.max_columns', None)

#disable warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from simulation_engine.algorithms.zaffalonbounds import ZaffalonBounds
from hydra import compose, initialize
from omegaconf import OmegaConf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
sim = BinaryConf._generate_data(
    n=500
)
dataConf = pd.DataFrame({'Y': sim['Y'], 'X': sim['X']})
dataConf

,Y,X
0,1,1
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
495,1,0
496,1,1
497,1,0
498,1,1


In [43]:
AutoBound.bound(query='ATE', 
                dagstring='X->Y, U -> X, U -> Y' ,
                data=dataConf, 
                observed=['X', 'Y'], 
                unob=['U'], 
                indep='X', 
                dep='Y')

(-0.442, 0.558)

In [29]:
dataMed = CustomBinaryDAG.generateMediation(500)
dataMed

,X,M,Y
0,1,0,0
1,0,0,0
2,1,1,0
3,0,1,1
4,1,1,0
...,...,...,...
495,1,0,1
496,1,1,1
497,1,1,0
498,1,0,1


In [ ]:
AutoBound.bound(query='ATE', 
                dagstring='X->Y, M->Y, U -> X, U -> Y, U -> M',
                data=dataMed, 
                observed=['X', 'Y', 'M'], 
                unob=['U'], 
                indep='X', 
                dep='Y')

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\tmari\AppData\Local\Temp\tmpw_ia_iei.glpk.raw --wglp C:\Users\tmari\AppData\Local\Temp\tmphqvftiw8.glpk.glp
 --cpxlp C:\Users\tmari\AppData\Local\Temp\tmpy0jnr8o4.pyomo.lp
Reading problem data from 'C:\Users\tmari\AppData\Local\Temp\tmpy0jnr8o4.pyomo.lp'...
10 rows, 33 columns, 113 non-zeros
186 lines were read
Writing problem data to 'C:\Users\tmari\AppData\Local\Temp\tmphqvftiw8.glpk.glp'...
202 lines were written
GLPK Simplex Optimizer 5.0
10 rows, 33 columns, 113 non-zeros
Preprocessing...
10 rows, 32 columns, 112 non-zeros
Scaling...
 A: min|aij| =  4.980e-01  max|aij| =  1.000e+00  ratio =  2.008e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 7
      0: obj =   9.913758620e-02 inf =   5.954e-02 (2)
LP HAS NO PRIMAL FEASIBLE SOLUTION
glp_simplex: unable to recover undefined or non-optimal solution
If you need actual output for non-optimal s

ERROR:pyomo.common.numeric_types:evaluating object as numeric value: objvar
    (object: <class 'pyomo.core.base.var.ScalarVar'>)
No value for uninitialized NumericValue object objvar


ValueError: No value for uninitialized NumericValue object objvar